In [1]:
import math
import itertools
import numpy as np
from math import sqrt, pi

import hoomd
import gsd.hoomd
import coxeter

#### Parameters

In [2]:
# integration timestep
dt=0.02045
# coupling
# kT=0.596161
kT=1.0
tau=100*dt

#### simultaion

In [3]:
device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device, seed=9920)

sim.create_state_from_gsd(filename='./DATA/init_config.gsd')
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT)

In [4]:
# system = hoomd.init.read_snapshot(snapshot)

nl_ex = hoomd.md.nlist.Cell(buffer=0.4, exclusions=['bond', '1-3', '1-4'])

lj = hoomd.md.pair.LJ(default_r_cut=12.0, nlist=nl_ex)
lj.params[('C', 'C')] = dict(epsilon=0.07, sigma=3.55)
lj.params[('H', 'H')] = dict(epsilon=0.03, sigma=2.42)
lj.params[('C', 'H')] = dict(epsilon=sqrt(0.07*0.03), sigma=sqrt(3.55*2.42))

coulomb = hoomd.md.long_range.pppm.make_pppm_coulomb_forces(nlist=nl_ex, 
                                                            resolution=(64,64,64), 
                                                            order=6, 
                                                            r_cut=12.0)

harmonic = hoomd.md.bond.Harmonic()
harmonic.params['CC'] = dict(k=2*268.0, r0=1.529)
harmonic.params['CH'] = dict(k=2*340.0, r0=1.09)

angle = hoomd.md.angle.Harmonic()
angle.params['CCC'] = dict(k=2*58.35, t0=112.7 * pi / 180)
angle.params['CCH'] = dict(k=2*37.5, t0=110.7 * pi / 180)
angle.params['HCH'] = dict(k=2*33.0, t0=107.8 * pi / 180)

dihedral = hoomd.md.dihedral.OPLS()
dihedral.params['CCCC'] = dict(k1=1.3, k2=-0.05, k3=0.2, k4=0.0)
dihedral.params['HCCC'] = dict(k1=0.0, k2=0.0, k3=0.3, k4=0.0)
dihedral.params['HCCH'] = dict(k1=0.0, k2=0.0, k3=0.3, k4=0.0)

lj_special_pairs = hoomd.md.special_pair.LJ()
lj_special_pairs.params['CCCC'] = dict(epsilon=0.07, sigma=3.55)
lj_special_pairs.r_cut['CCCC']=12.0
lj_special_pairs.params['HCCH'] = dict(epsilon=0.03, sigma=2.42)
lj_special_pairs.r_cut['HCCH']=12.0
lj_special_pairs.params['HCCC'] = dict(epsilon=sqrt(0.07 * 0.03),
                                       sigma=sqrt(3.55*2.42))
lj_special_pairs.r_cut['HCCC']=12.0

coulomb_special_pairs = hoomd.md.special_pair.Coulomb()
coulomb_special_pairs.params['CCCC'] = dict(alpha=0.5)
coulomb_special_pairs.r_cut['CCCC'] = 12.0
coulomb_special_pairs.params['HCCC'] = dict(alpha=0.5)
coulomb_special_pairs.r_cut['HCCC'] = 12.0
coulomb_special_pairs.params['HCCH'] = dict(alpha=0.5)
coulomb_special_pairs.r_cut['HCCH'] = 12.0

#### thermodynamic ensemble

In [5]:
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau)

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/md/methods/methods.py:102: FutureWarning: NVT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantVolume method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


#### integrator

In [6]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[nvt],
                                 forces=[lj, 
                                         coulomb[0],
                                         coulomb[1],
                                         harmonic,
                                         angle,
                                         dihedral,
                                         lj_special_pairs, 
                                         coulomb_special_pairs])

#### writer

In [7]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(thermodynamic_properties)

In [8]:
gsd_writer = hoomd.write.GSD(filename='./DATA/equilibriate.gsd',
                             trigger=hoomd.trigger.Periodic(10),
                             log=logger,
                             mode='wb')

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x11bc5f220> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


In [9]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run

In [ ]:
sim.run(1e5)

notice(2): charge.pppm: RMS error: 1.29239e-08
